In [7]:
import os
import numpy as np
import csv
import pandas as pd
os.chdir('/Users/renalkakhan/Documents/GitHub/CS598_DLH_Project/')
from dataset.preprocessing.tf_idf_all_feature_matrix_gen import TFIDFFeatureGeneration
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier

In [8]:
class DecisionTree:
    def __init__(self, x_train, y_train, x_test, y_test, k):
        self.dtc = DecisionTreeClassifier(splitter='random', random_state=42)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.k = k
        # print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
        
    def feature_selection_SelectKBest(self):
        k_best = SelectKBest(chi2, k=self.k)
        k_best.fit(self.x_train, self.y_train)
        self.x_train = k_best.transform(self.x_train)
        self.x_test = k_best.transform(self.x_test)
        
    def feature_selection_ExtraTreesClassifier(self):
        clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
        clf.fit(self.x_train, self.y_train)
        importances = clf.feature_importances_
        indices = np.argsort(importances)[::-1]
        self.x_train = self.x_train[:, indices[:self.k]]
        self.x_test = self.x_test[:, indices[:self.k]]

    def train(self):
        self.dtc.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.dtc.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        #print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [9]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']

column_headings = ["Morbidity Class", "DT_Macro F1", "DT_Micro F1"]
    

In [10]:
with open("./results/tf-idf-features/performance_DT_AllFeatures.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 0)
        dt_obj.train()

        f1_macro, f1_micro = dt_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf-features/performance_DT_AllFeatures.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/tf-idf-features/performance_DT_AllFeatures.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Macro F1 score: 0.9093077411324556 and Micro F1 Score 0.9649425287356321
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
Macro F1 score: 0.8389718527222257 and Micro F1 Score 0.844949494949495
CHF
(243, 600) (243,) Counter({1.0: 243})
Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Macro F1 score: 0.7401403277115528 and Micro F1 Score 0.8333722969023963
Diabetes
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Macro F1 score: 0.880859716288606 and Micro F1 Score 0.9029135338345865
Gallstones
(593, 600) (593,) Counter({0.0: 506, 1.0: 87})
Macro F1 score: 0.47352767430103404 and Micro F1 Score 0.7048022598870056
GERD
(487, 600) (487,) Counter({0.0: 372, 1.0: 115})
Macro F1 score: 0.5507310688168865 and Micro F1 Score 0.6593962585034013
Gout
(596, 600) (596,) Counter({0.0: 518, 1.0: 78})
Macro F1 score: 0.5287721360400575 and Micro F1 Score 0.7666666666666668
Hypercholesterolemia

In [11]:
with open("./results/tf-idf-features/performance_DT_SelectKBest.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 100)
        dt_obj.feature_selection_SelectKBest()
        dt_obj.train()   

        f1_macro, f1_micro = dt_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf-features/performance_DT_SelectKBest.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/tf-idf-features/performance_DT_SelectKBest.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Macro F1 score: 0.9010978739079999 and Micro F1 Score 0.9579854809437387
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
Macro F1 score: 0.8626044341711383 and Micro F1 Score 0.8686195286195286
CHF
(243, 600) (243,) Counter({1.0: 243})
Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Macro F1 score: 0.7010170070710479 and Micro F1 Score 0.7989772063120982
Diabetes
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Macro F1 score: 0.8783145757928699 and Micro F1 Score 0.9030075187969924
Gallstones
(593, 600) (593,) Counter({0.0: 506, 1.0: 87})
Macro F1 score: 0.5270960255950692 and Micro F1 Score 0.7656779661016949
GERD
(487, 600) (487,) Counter({0.0: 372, 1.0: 115})
Macro F1 score: 0.5423708566287758 and Micro F1 Score 0.6794642857142857
Gout
(596, 600) (596,) Counter({0.0: 518, 1.0: 78})
Macro F1 score: 0.5596387959665803 and Micro F1 Score 0.8019774011299434
Hypercholesterolemi

In [12]:
with open("./results/tf-idf-features/performance_DT_ExtraTreesClassifier.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 100)
        dt_obj.feature_selection_ExtraTreesClassifier()
        dt_obj.train()

        f1_macro, f1_micro = dt_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf-features/performance_DT_ExtraTreesClassifier.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/tf-idf-features/performance_DT_ExtraTreesClassifier.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Macro F1 score: 0.9013072243691992 and Micro F1 Score 0.9580459770114944
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
Macro F1 score: 0.86690570353518 and Micro F1 Score 0.8723905723905723
CHF
(243, 600) (243,) Counter({1.0: 243})
Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Macro F1 score: 0.6872814521157056 and Micro F1 Score 0.7921683226183518
Diabetes
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Macro F1 score: 0.8589442345811644 and Micro F1 Score 0.8818922305764412
Gallstones
(593, 600) (593,) Counter({0.0: 506, 1.0: 87})
Macro F1 score: 0.5153562304511325 and Micro F1 Score 0.7621468926553672
GERD
(487, 600) (487,) Counter({0.0: 372, 1.0: 115})
Macro F1 score: 0.5227570098307155 and Micro F1 Score 0.657312925170068
Gout
(596, 600) (596,) Counter({0.0: 518, 1.0: 78})
Macro F1 score: 0.5150961734379089 and Micro F1 Score 0.7936158192090396
Hypercholesterolemia
(